<a href="https://colab.research.google.com/github/41371120h/PL-Repo.peng/blob/main/41371120H_%E6%94%B6%E6%94%AF%E7%AE%A1%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
import gradio as gr

In [8]:
# Google Sheets 認證
creds, _ = default()
gc = gspread.authorize(creds)

# 開啟試算表
gsheets = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1T_CYo5tydoyyBj8LfMpQCcAD4IJqnNkczZb_6WIslZk/edit?gid=0#gid=0'
)

# 取得或建立 worksheet
try:
    worksheet = gsheets.worksheet('收支紀錄')
except:
    worksheet.append_row(['日期', '項目名稱', '收入', '支出', '請款人', '支付方式', '是否已給錢', '結餘'])

In [9]:
def add_record(date_str, item_str, in_or_out, amount, payer, payment_method, other_method, paid_status):
    # 如果選 "其他"，就用 other_method 當支付方式
    if payment_method == "其他" and other_method.strip() != "":
        payment_method = other_method.strip()

    # 判斷收入/支出
    if in_or_out == "收入":
        income = float(amount)
        expense = 0
    else:
        expense = float(amount)
        income = 0

    # 新增資料
    new_row = [date_str, item_str, income, expense, payer, payment_method, paid_status, ""]
    worksheet.append_row(new_row)

    # 更新 DataFrame
    records = worksheet.get_all_records()
    df = pd.DataFrame(records)
    df.columns = df.columns.str.strip()
    df['收入'] = pd.to_numeric(df.get('收入', 0), errors='coerce').fillna(0)
    df['支出'] = pd.to_numeric(df.get('支出', 0), errors='coerce').fillna(0)

    # 計算累積結餘
    df['結餘'] = (df['收入'] - df['支出']).cumsum()

    # 更新試算表結餘欄位
    cell_list = worksheet.range(f"H2:H{len(df)+1}")
    for i, cell in enumerate(cell_list):
        cell.value = int(df.iloc[i]['結餘'])
    worksheet.update_cells(cell_list)

    # 計算總結
    total_income = df['收入'].sum()
    total_expense = df['支出'].sum()
    balance = df['結餘'].iloc[-1] if not df.empty else 0

    summary = f"✅ 已新增一筆紀錄！\n\n"
    summary += f"📌 項目: {item_str}\n💰 收入: {income}\n💸 支出: {expense}\n👤 請款人: {payer}\n"
    summary += f"💳 支付方式: {payment_method}\n📅 日期: {date_str}\n\n"
    summary += "--- 收支總結 ---\n"
    summary += f"總收入: {total_income:,.0f}\n"
    summary += f"總支出: {total_expense:,.0f}\n"
    summary += f"目前結餘: {balance:,.0f}\n"

    # 未付款項目
    unpaid_msg = "\n--- 未付款項目 ---\n"
    if '是否已給錢' in df.columns:
        unpaid = df[df['是否已給錢'].str.contains("否", na=False)]
        if not unpaid.empty:
            unpaid_msg += unpaid[['日期', '項目名稱', '支出', '請款人', '支付方式']].to_string(index=False)
        else:
            unpaid_msg += "✅ 沒有未付款項目"
    else:
        unpaid_msg += "❌ 試算表沒有「是否已給錢」欄位"

    return summary + "\n" + unpaid_msg

In [10]:
# Gradio 介面
with gr.Blocks() as demo:
    gr.Markdown("# 💵 收支管理系統（Google Sheets 版）")

    with gr.Row():
        date_str = gr.Textbox(label="日期 (YYYY-MM-DD)", value=datetime.today().strftime("%Y-%m-%d"))
        item_str = gr.Textbox(label="項目名稱")

    with gr.Row():
        in_or_out = gr.Radio(["收入", "支出"], label="收支類型", value="支出")
        amount = gr.Number(label="金額", value=0)

    with gr.Row():
        payer = gr.Textbox(label="請款人")
        payment_method = gr.Dropdown(choices=["現金", "信用卡", "Line Pay", "其他"], label="支付方式", value="現金")
        other_method = gr.Textbox(label="若選擇其他，請輸入方式")
        paid_status = gr.Radio(["是", "否"], label="是否已給錢", value="是")

    submit_btn = gr.Button("新增紀錄")
    output = gr.Textbox(label="結果", lines=15)

    submit_btn.click(
        fn=add_record,
        inputs=[date_str, item_str, in_or_out, amount, payer, payment_method, other_method, paid_status],
        outputs=output
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ab7289bcc80d0e43c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
